# [M_04] PROTOKÓŁ: PAMIĘĆ DŁUGOTRWAŁA (QDRANT RAG)

**PROJEKT:** OMNI-OPERATOR-V1  
**SILNIK:** GEMINI 2.5 FLASH + TEXT-EMBEDDING-004  
**STATUS:** IMPLEMENTACJA_RECOGNITION

Ten moduł odpowiada za budowę "pamięci strategicznej" systemu. Zapisujemy wyniki analiz (M_01) oraz wygenerowane posty (M_02) w wektorowej bazie danych **Qdrant**.

**Dlaczego to robimy?**
1. **Analiza Stylu:** System może sprawdzić, jakie hooki stosował w przeszłości.
2. **Unikanie Powtórzeń:** Agent wie, o czym już pisał, aby nie powielać treści.
3. **Sovereign Data:** Twoja wiedza o tym, co "działa", zostaje na Twoim serwerze w Dockerze.

In [2]:
import os
import sys
import uuid
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import google.generativeai as genai

# 1. KOREKTA ŚCIEŻKI ROBOCZEJ
if os.getcwd().endswith("notebooks"):
    os.chdir("..")

# Dodanie src do path
sys.path.append(os.path.join(os.getcwd(), "src"))

from src.core.config import settings

# 2. INICJALIZACJA KLIENTÓW
# Qdrant działa na porcie 6333 w Twoim Dockerze
qclient = QdrantClient(url=settings.qdrant_url)

# Konfiguracja Google do Embeddingów (zamiana tekstu na wektory)
genai.configure(api_key=settings.gemini_api_key)

print(f"LOG: System pamięci gotowy. Połączono z Qdrant na {settings.qdrant_url}")

c:\Users\takze\OneDrive\Pulpit\project\omni-operator-v1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\takze\AppData\Local\Temp\ipykernel_36992\930723068.py:6: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


LOG: System pamięci gotowy. Połączono z Qdrant na http://localhost:6333


C:\Users\takze\AppData\Local\Temp\ipykernel_36992\930723068.py:19: UserWarning: Qdrant client version 1.16.2 is incompatible with server version 1.12.0. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  qclient = QdrantClient(url=settings.qdrant_url)


## 1. Tworzenie Kolekcji (Schema bazy wektorowej)

Definiujemy kolekcję `content_memory`. Rozmiar wektora (768) odpowiada najnowszemu modelowi Google `text-embedding-004`.

In [3]:
COLLECTION_NAME = "content_memory"

def init_memory():
    """Tworzy kolekcję w Qdrant, jeśli jeszcze nie istnieje."""
    collections = qclient.get_collections().collections
    exists = any(c.name == COLLECTION_NAME for c in collections)
    
    if not exists:
        print(f"LOG: Tworzę nową kolekcję: {COLLECTION_NAME}")
        qclient.create_collection(
            collection_name=COLLECTION_NAME,
            vectors_config=VectorParams(size=768, distance=Distance.COSINE),
        )
        print("✅ KOLEKCJA UTWORZONA")
    else:
        print(f"LOG: Kolekcja {COLLECTION_NAME} już istnieje.")

init_memory()

LOG: Kolekcja content_memory już istnieje.


## 2. Funkcja Osadzania (Embedding)

Zamieniamy ludzki tekst na listę liczb (wektor), którą AI potrafi porównywać matematycznie.

In [4]:
def get_embedding(text: str):
    """Generuje wektor dla podanego tekstu przy użyciu modelu Google."""
    result = genai.embed_content(
        model="models/text-embedding-004",
        content=text,
        task_type="retrieval_document"
    )
    return result['embedding']

# TEST:
sample_vec = get_embedding("Inżynieria AI w Kuźni Operatorów")
print(f"LOG: Wygenerowano wektor o długości: {len(sample_vec)}")

LOG: Wygenerowano wektor o długości: 768


## 3. Zapisywanie Kampanii do Pamięci

Implementujemy logikę "zapamiętywania". Zapisujemy treść kampanii wraz z jej metadanymi (temat, platformy).

In [5]:
from datetime import datetime

def save_campaign_to_memory(brief_data: dict, topic: str):
    """Zapisuje raport kampanii do bazy Qdrant."""
    
    print(f"LOG: Generowanie embeddingu dla tematu: {topic}...")
    # 1. Pobieramy wektor
    content_to_embed = f"Topic: {topic}. Strategy: {brief_data['overall_strategy']}"
    vector = get_embedding(content_to_embed)
    
    # 2. Przygotowujemy punkt danych
    point_id = str(uuid.uuid4())
    
    # Używamy natywnego datetime zamiast os.popen('date')
    timestamp_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    print(f"LOG: Wysyłanie do Qdrant (ID: {point_id})...")
    
    # 3. Zapis punktu do bazy
    qclient.upsert(
        collection_name=COLLECTION_NAME,
        points=[
            PointStruct(
                id=point_id,
                vector=vector,
                payload={
                    "topic": topic,
                    "strategy": brief_data['overall_strategy'],
                    "type": "campaign_brief",
                    "timestamp": timestamp_str
                }
            )
        ]
    )
    print(f"✅ ZAPAMIĘTANO POMYŚLNIE: {topic}")

# TEST: Ponowne uruchomienie
mock_brief = {
    "overall_strategy": "Pozycjonowanie Takzen Dev jako lidera suwerennego AI.",
    "posts": []
}
save_campaign_to_memory(mock_brief, "Suwerenność AI 2026")

LOG: Generowanie embeddingu dla tematu: Suwerenność AI 2026...
LOG: Wysyłanie do Qdrant (ID: dcb746ce-16e0-4c16-8be1-56007ccaf4a8)...
✅ ZAPAMIĘTANO POMYŚLNIE: Suwerenność AI 2026


## 4. Wyszukiwanie Semantyczne (RAG Test)

Sprawdzamy, czy system potrafi odnaleźć powiązane treści bez użycia słów kluczowych, bazując na samym "sensie" zapytania.

In [7]:
def search_memory(query: str, limit: int = 2):
    """
    Przeszukuje pamięć przy użyciu nowoczesnego API Qdrant 1.16+.
    """
    print(f"LOG: Generowanie embeddingu dla zapytania: '{query}'...")
    query_vector = get_embedding(query)
    
    print("LOG: Przeszukiwanie bazy wektorowej...")
    
    # Używamy najnowszego API query_points (Standard 2025/2026)
    hits = qclient.query_points(
        collection_name=COLLECTION_NAME,
        query=query_vector,
        limit=limit,
        with_payload=True
    ).points
    
    if not hits:
        print("ℹ️ Brak pasujących wspomnień w bazie.")
        return

    print(f"\n🔎 WYNIKI DLA: '{query}'")
    print("-" * 40)
    for hit in hits:
        score = hit.score
        topic = hit.payload.get('topic', 'Brak tematu')
        strategy = hit.payload.get('strategy', '')[:100]
        print(f" -> [Zgodność: {score:.2f}] {topic}")
        print(f"    Strategia: {strategy}...\n")

# TEST OPERACYJNY:
search_memory("Szukam czegoś o niezależności od SaaS")

LOG: Generowanie embeddingu dla zapytania: 'Szukam czegoś o niezależności od SaaS'...
LOG: Przeszukiwanie bazy wektorowej...

🔎 WYNIKI DLA: 'Szukam czegoś o niezależności od SaaS'
----------------------------------------
 -> [Zgodność: 0.74] Suwerenność AI 2026
    Strategia: Pozycjonowanie Takzen Dev jako lidera suwerennego AI....



## STATUS: MODUŁ 04 ZAKOŃCZONY

Mamy w pełni funkcjonalną pamięć wektorową. Twój system potrafi teraz gromadzić doświadczenie.

**Osiągnięcia:**
1. Integracja lokalnego Qdranta z Embeddingami Google.
2. Możliwość zapisu i odzyskiwania wiedzy (RAG).
3. Podwaliny pod Etap 5 (Dyrygent), gdzie system będzie sprawdzał bazę przed każdym montażem.